In [1]:
import numpy as np
import pandas as pd
import healpy as hp
from statistics import stdev, mean

nside = 8   # this is for masking 1 and masking 2
nf = 100  # number of samples

# Compute Rényi entropies for q = 1 to 5

def compute_renyi_entropy(input_file, output_file,rmin, rmax, Nside= nside, nbin=30):
    Npix = hp.nside2npix(Nside)

    # Load data
    df = pd.read_csv(input_file, sep="\t", header=None)
    df.columns = ['r', 'th', 'ph']
    r1 = df['r'].to_numpy()
    th1 = df['th'].to_numpy()
    ph1 = df['ph'].to_numpy()


    #rmin, rmax = np.min(r1), np.max(r1)
    dr = (rmax - rmin) / nbin

    m = np.zeros((nbin, Npix), dtype=float)
    Neff = np.zeros(nbin, dtype=int)

    for rr, tt, pp in zip(r1, th1, ph1):
        for j in range(nbin):
            if rmin <= rr <= (rmin + (j+1) * dr):
                px = hp.ang2pix(Nside, tt, pp)
                m[j][px] += 1
                Neff[j] += 1

    p = np.zeros((nbin, Npix), dtype=float)
    for i in range(nbin):
        for px in range(Npix):
            if Neff[i] > 0:
                p[i][px] = m[i][px] / Neff[i]

    H = np.zeros((nbin, 5), dtype=float)
    a = np.zeros((nbin, 5), dtype=float)
    
    for k in range(5):
        h = np.zeros(nbin, dtype=float)
        q = k + 1
        if q == 1:
            for i in range(nbin):
                for px in range(Npix):
                    if p[i][px] > 0:
                        H[i][k] -= p[i][px] * np.log10(p[i][px])
        else:
            for i in range(nbin):
                for px in range(Npix):
                    h[i] += p[i][px] ** q
                if h[i] > 0:
                    H[i][k] = np.log10(h[i]) / (1 - q)

        for i in range(nbin):
            a[i][k] = H[i][k]

    R = np.zeros(nbin)
    for b in range(nbin):
        R[b] = rmin + (b+1)*dr
        
    df1=pd.DataFrame(data=a)
    df2=pd.DataFrame(data=R)
    df3= pd.concat([df2,df1], axis=1, join='inner')
    df3.to_csv(output_file, sep='\t', header=False, index=False)

###########################################################################

R_max = np.zeros(3)
R_min = np.zeros(3)

for n in range(3):
    f_in = '../data_prep/mask1/masked_sample_' + str(n+1) + '.dat'
    # f_in = '../data_prep/mask2/masked_sample_' + str(n+1) + '.dat'
    
    df = pd.read_csv(f_in,sep="\t",header = None)
    df.columns = ['r','th','ph']
    R_max[n] = df['r'].max()
    R_min[n] = df['r'].min()

    f_out1 = 'renyi_anis_s'+ str(n+1)+'.dat'
    compute_renyi_entropy(f_in, f_out1,R_min[n],R_max[n])
    
    
    for l in range(nf):
        fd=df.sample(frac=0.8,replace=True,random_state = l)
        f_samp = 'sample'+ str(n+1) + '_' + str(l+1) + '.dat' 
        fd.to_csv(f_samp,sep="\t",header = None,index = False)
        f_out2 = 'anis_s'+ str(n+1) + '_' + str (l+1) + '.dat'
        compute_renyi_entropy(f_samp,f_out2,R_min[n],R_max[n])

In [2]:
# calculation of normalized entropy dispersion and error associated with it

for n in range(3):
    file_in = 'renyi_anis_s'+ str(n+1)+'.dat'
    data = np.loadtxt(file_in)
    r = data[:, 0]
    samples = data[:, 1:6]

    s_mean = np.mean(samples, axis=1)
    stab_cri = np.std(samples, axis=1, ddof=0)
    frac_cri = stab_cri / s_mean

    df = pd.DataFrame({'r': r, 'crit': frac_cri})
    file_out = 'sample'+ str(n+1)+'_crit.csv'
    df.to_csv(file_out, index=False)


for n in range(3):
    df_crit = pd.read_csv('sample'+ str(n+1)+'_crit.csv')
    r = df_crit['r'].to_numpy()
    crit_mean = df_crit['crit'].to_numpy()
    nbin = len(r)

    criteria = np.zeros((nf, nbin))

    for f in range(nf):
        file = 'anis_s'+ str(n+1) + '_' + str (f+1) + '.dat'
        data = np.loadtxt(file)
        samples = data[:, 1:6]
        mean_vals = np.mean(samples, axis=1)
        stab_vals = np.std(samples, axis=1, ddof=0)
        criteria[f] = stab_vals / mean_vals

    # Compute std across samples at each radius bin
    crit_std = np.std(criteria, axis=0, ddof=1)

    df_out = pd.DataFrame({'r': r, 'crit': crit_mean, 'sd': crit_std})
    df_out.to_csv('sample_'+str(n+1)+'_criteria_err.csv' , index=False)


In [3]:
import os
for n in range(3):
    f_out1 = 'renyi_anis_s'+ str(n+1)+'.dat'
    os.remove(f_out1)
    f_out2 = 'sample'+ str(n+1)+'_crit.csv'
    os.remove(f_out2)
    for l in range(nf):
        file_name = 'anis_s'+ str(n+1) + '_' + str (l+1) + '.dat'
        os.remove(file_name)
        f_samp = 'sample'+ str(n+1) + '_' + str(l+1) + '.dat' 
        os.remove(f_samp)
